![](https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQUWXStMee9eQjoH2a3K8qslB6z_PnVU5GEbQ&usqp=CAU)inet.ox.ac.uk

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px
import plotly.offline as py
from plotly import tools, subplots
py.init_notebook_mode(connected=True)
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('../input/hackathon/task_2-owid_covid_data-21_June_2020.csv')
df.head()

#Codes from Crislânio Macedo https://www.kaggle.com/caesarlupum/brazil-against-the-advance-of-covid-19/notebook?select=h2oai-pystacknet-af571e0 Master and friend

In [ ]:
# Data processing, metrics and modeling
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from datetime import datetime
from sklearn.metrics import precision_score, recall_score, confusion_matrix, accuracy_score, roc_auc_score, f1_score, roc_curve, auc,precision_recall_curve
from sklearn import metrics
# Lgbm
import lightgbm as lgb
import catboost
from catboost import Pool
import xgboost as xgb

# Suppr warning
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def missing_data(data):
    total = data.isnull().sum()
    percent = (data.isnull().sum()/data.isnull().count()*100)
    tt = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    types = []
    for col in data.columns:
        dtype = str(data[col].dtype)
        types.append(dtype)
    tt['Types'] = types
    return(np.transpose(tt))

In [ ]:
%%time
missing_data(df)

In [ ]:
import missingno as msno

In [ ]:
#Nullity Matrix. The msno.matrix nullity matrix is a data-dense display which lets you quickly visually analyse data completion.
msno.matrix(df.head(10000))

#Heatmap. The missingno correlation heatmap measures nullity correlation: how strongly the presence or absence of one variable affects the presence of another:
Don't write anything before the per cent time, Otherwise it wouldn't work.

In [ ]:
%%time
a = msno.heatmap(df, sort='ascending')
a

#Dendrogram.The dendrogram allows you to more fully correlate variable completion, revealing trends deeper than the pairwise ones visible in the correlation heatmap:

In [ ]:
%%time
a2 = msno.dendrogram(df)
a2

In [ ]:
# Number of unique classes in each object column
df.select_dtypes('object').apply(pd.Series.nunique, axis = 0)

#Correlations

In [ ]:
# Find correlations with the target and sort
correlations = df.corr()['total_cases'].sort_values()

# Display correlations
print('Most Positive Correlations:\n', correlations.tail(15))
print('\nMost Negative Correlations:\n', correlations.head(15))

In [ ]:
%%time
features = df.columns.values[2:112]
corrs_ = df[features].corr().abs().unstack().sort_values(kind="quicksort").reset_index()
corrs_ = corrs_[corrs_['level_0'] != corrs_['level_1']]
corrs_.head(10)

The least correlated features.

In [ ]:
corrs_.head(10)

The top most correlated features, besides the same feature pairs.

In [ ]:
corrs_.tail(10)

#Correlations clustermap

In [ ]:
%%time
corrs = df.corr()
plt.figure(figsize = (10, 6))
# Heatmap of correlations
sns.heatmap(corrs, cmap = plt.cm.RdYlBu_r, vmin = -0.25, annot = False, vmax = 0.8)
plt.title('Clustermap');

In [ ]:
def plot_dist_col(column):
    pos__df = df[df['total_cases'] ==1]
    neg__df = df[df['total_cases'] ==0]

    '''plot dist curves for train and test weather data for the given column name'''
    fig, ax = plt.subplots(figsize=(8, 8))
    sns.distplot(pos__df[column].dropna(), color='green', ax=ax).set_title(column, fontsize=16)
    sns.distplot(neg__df[column].dropna(), color='purple', ax=ax).set_title(column, fontsize=16)
    plt.xlabel(column, fontsize=15)
    plt.legend(['total_cases', 'total_tests'])
    plt.show()
plot_dist_col('total_tests')

#Label Encoding.

It assigns each unique value to a different integer.

In [ ]:
#fill in mean for floats
for c in df.columns:
    if df[c].dtype=='float16' or  df[c].dtype=='float32' or  df[c].dtype=='float64':
        df[c].fillna(df[c].mean())

#fill in -999 for categoricals
df = df.fillna(-999)
# Label Encoding
for f in df.columns:
    if df[f].dtype=='object': 
        lbl = LabelEncoder()
        lbl.fit(list(df[f].values))
        df[f] = lbl.transform(list(df[f].values))
        
print('Labelling done.') 

#Codes from Serigne https://www.kaggle.com/serigne/stacked-regressions-top-4-on-leaderboard/notebook

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
        
    # we define clones of the original models to fit the data in
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]
        
        # Train cloned base models
        for model in self.models_:
            model.fit(X, y)

        return self
    
    #Now we do the predictions for cloned models and average them
    def predict(self, X):
        predictions = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        return np.mean(predictions, axis=1)

In [ ]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import make_pipeline
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))

In [ ]:
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)

In [ ]:
from sklearn.kernel_ridge import KernelRidge
KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)

In [ ]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))

In [ ]:
from sklearn.model_selection import KFold, cross_val_score, train_test_split
#Validation function
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(df.values)
    rmse= np.sqrt(-cross_val_score(model, df.values, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

In [ ]:
#ntrain = train.shape[0]
#ntest = test.shape[0]
y_train = df.total_cases.values
#all_data = pd.concat((train, test)).reset_index(drop=True)
a#ll_data.drop(['SalePrice'], axis=1, inplace=True)
#print("all_data size is : {}".format(all_data.shape))

I commented the snippet below because it was frozen then I couldn't perform anything else. I made "Stacked Stucked OWID" public it's showing like some "inutility script" all black. I worked on it for 3 days, trying to use PyStackNet then Stack. The worst, it only appears in my page though it's Public. It doesn't appear in the list of that Dataset. I simply don't know why. No clue. Maybe it was abducted and I was stucked on it. The "fade away" from the title is due to that situation. (The 1st attempt to Stack isn't listed in the Datasets challenge list, simply remains in my page, as a sign of shame. 

In [ ]:
#averaged_models = AveragingModels(models = (ENet, GBoost, KRR, lasso))

#score = rmsle_cv(averaged_models)
#print(" Averaged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

Das War's, Kaggle Notebook Runner: Marília Prata  @mpwolke.